<div class="alert alert-danger">
    <h4 style="font-weight: bold; font-size: 28px;">Extreme Gradient Boosting with Selected Features</h4>
    <h5 style="font-weight: bold; font-size: 24px;">Hyperparameter Tuning using Expanding Window</h5>
    <p style="font-size: 20px;">NBA API Seasons 2021-22 to 2023-24</p>
</div>

<a name="Models"></a>

# Table of Contents

[Setup](#Setup)

[Data](#Data)

[Inspect Expanding Training Window](#Inspect-Training-Windows)

[Functions](#Functions)

**[1. Target: Total Points (over / under)](#1.-Target:-Total-Points-(over-/-under))**
  
**[2. Target: Difference in Points (plus / minus)](#2.-Target:-Difference-in-Points-(plus-/-minus))**

**[3. Target: Game Winner (moneyline)](#3.-Target:-Game-Winner-(moneyline))**

# Setup

[Return to top](#Models)

In [1]:
import sys
from pathlib import Path
# get current working directory
cwd = %pwd
# add shared_code directory to Python sys.path
sys.path.append(str(Path(cwd).parent / "shared_code"))
# import all libraries in shared_code directory 'imports.py' file
from imports import *
%matplotlib inline

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


# Data

[Return to top](#Models)

Data splits:

- Define NBA Season 2021-22 as the TRAINING set: regular season is 2021-10-19 to 2022-04-10. 
- Define NBA Season 2022-23 as the VALIDATION set: regular season is 2022-10-18 to 2023-04-09.
- Define NBA Season 2023-24 as the TESTING set: regular season is 2023-10-24 to 2024-04-14.

In [2]:
# load, filter (by time) and scale data
pts_all_features, pm_all_features, res_all_features, test_set_obs = utl.load_and_scale_data(
    input_data='../../data/processed/nba_team_matchups_all_rolling_stats_merged_2021_2024_r05.csv',
    seasons_to_keep=['2021-22', '2022-23', '2023-24'],
    training_season='2021-22',
    feature_prefixes=['ROLL_', 'ROLLDIFF_'],
    scaler_type='minmax', 
    scale_target=False
)

Season 2021-22: 1186 games
Season 2022-23: 1181 games
Season 2023-24: 691 games
Total number of games across sampled seasons: 3058 games


In [3]:
# define number of games in seasons
season_22_ngames = 1186
season_23_ngames = 1181
season_24_ngames = 691

In [4]:
# load the best features dictionaries back from the file
with open('../../data/selected_features/feature_set_01_filter_and_wrapper.json', 'r') as json_file:
    selected_features_filter_and_wrapper = json.load(json_file)

with open('../../data/selected_features/feature_set_02_embedded.json', 'r') as json_file:
    selected_features_embedded = json.load(json_file)

In [5]:
# subset the features
pts_sub_fw_features = pts_all_features[selected_features_filter_and_wrapper['TOTAL_PTS'] + ['TOTAL_PTS']]
pts_sub_e_features = pts_all_features[selected_features_embedded['TOTAL_PTS'] + ['TOTAL_PTS']]

pm_sub_fw_features = pm_all_features[selected_features_filter_and_wrapper['PLUS_MINUS'] + ['PLUS_MINUS']]
pm_sub_e_features = pm_all_features[selected_features_embedded['PLUS_MINUS'] + ['PLUS_MINUS']]

res_sub_fw_features = res_all_features[selected_features_filter_and_wrapper['GAME_RESULT'] + ['GAME_RESULT']]
res_sub_e_features = res_all_features[selected_features_embedded['GAME_RESULT'] + ['GAME_RESULT']]

In [6]:
pts_sub_fw_features.head()

,ROLL_HOME_FTM,ROLL_HOME_OPP_PTS_PAINT,ROLL_HOME_PTS_PAINT,ROLL_AWAY_PTS,ROLL_HOME_PTS_FB,ROLL_AWAY_OFF_LOOSE_BALLS_RECOVERED,ROLL_AWAY_DEF_BOXOUTS,ROLL_AWAY_estimatedPace,ROLL_AWAY_PTS_PAINT,ROLL_AWAY_DFG_PCT,ROLL_HOME_AST,ROLL_AWAY_assistToTurnover,ROLL_HOME_OPP_TOV_PCT,ROLL_HOME_estimatedDefensiveRating,TOTAL_PTS
GAME_DATE,,,,,,,,,,,,,,,
2021-10-23,0.805,0.808,0.50,0.577,1.000,0.60,0.208,0.443,0.391,1.000,0.612,0.291,0.840,0.298,185
2021-10-23,0.466,0.758,0.25,0.096,0.581,0.30,0.307,0.522,0.000,0.453,0.000,0.161,0.762,0.176,198
2021-10-23,0.593,0.505,0.70,0.635,0.065,0.75,0.208,0.465,0.565,0.023,0.561,0.589,0.144,0.717,239
2021-10-23,0.297,0.606,0.70,0.250,0.839,0.30,0.109,0.691,0.261,0.965,0.918,0.218,0.519,0.328,232
2021-10-24,0.254,0.202,0.70,1.000,0.548,0.45,0.455,0.646,0.348,0.732,0.765,0.393,0.591,0.002,204


# Inspect Expanding Training Window

[Return to top](#Models)

In [7]:
# expanding window configuration
initial_train_size = 10  # starting size of the training set
test_size = 1            # leave-one-out (LOO) cross-validation
gap_size=0               # should there be a gap between train and test sets?
expansion_limit=None     # the limit on the test set observations

counter = 0
max_splits_to_show = 15

# show first few splits
for train_indices, test_indices in utl.expanding_window_ts_split(pts_all_features, initial_train_size, 
                                                                 test_size=test_size, gap_size=gap_size,
                                                                 expansion_limit=expansion_limit):
    print("TRAIN:", train_indices, "TEST:", test_indices)
    counter += 1
    if counter >= max_splits_to_show:
        break

TRAIN: [0 1 2 3 4 5 6 7 8 9] TEST: [10]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10] TEST: [11]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11] TEST: [12]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12] TEST: [13]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13] TEST: [14]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14] TEST: [15]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] TEST: [16]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16] TEST: [17]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17] TEST: [18]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18] TEST: [19]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] TEST: [20]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20] TEST: [21]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21] TEST: [22]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22] TEST: [23]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9

# Functions

[Return to top](#Models)

For some idiotic reason 'XGBoost' requires the training function to be in the notebook rather than imported, at least while using early stopping. Also, we need to pass in validation set data directly to the `.fit` method, which requires a slight modification to the function in the `utl` library.

In [8]:
def train_with_expanding_window(df, initial_train_size, test_size, target_col, model, gap_size=0,
                                expansion_limit=None, fitted_model=False, ensure_diversity=False):
    """
    Trains a given model using an expanding window approach on a specified DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the features and target variable.
    - initial_train_size (int): The initial size of the training dataset.
    - test_size (int): The size of the test dataset for each split, typically 1 for LOO CV.
    - target_col (str): The name of the target column in `df`.
    - model (model object): The instantiated model to be trained, e.g., LinearRegression() or LogisticRegression().
    - gap_size (int): The gap size between training and test datasets. Default is 0.
    - expansion_limit (int, optional): The maximum number of times the training set is expanded by 1 observation during the expanding window process. This parameter controls the total number of train-test splits generated, indirectly determining the final size of the training set. If set, the training process will stop once this limit is reached, potentially leaving some data unused. If None, the training set will expand until all but the last observation are used for training.
    - fitted_model (bool): whether to return the fitted model instance.
    - ensure_diversity (bool, optional): For logistic regression, ensures the initial training data includes both classes. Default is False.

    Returns:
    - model_outputs (list): A list of model predictions or probabilities for the test sets across all splits.
    - y_true (list): A list of the actual target values corresponding to each prediction in `model_outputs`.

    This function iterates over the dataset using an expanding window to create training and test splits, 
    trains the specified `model` on each training split, and stores the model's predictions or probabilities.
    """
    import time
    
    start_time = time.time()

    # initialize storage for model outputs and true labels
    model_outputs = []  # store predictions or probabilities
    y_true = []

    for train_indices, test_indices in utl.expanding_window_ts_split(
        df, initial_train_size, test_size=test_size, gap_size=gap_size,
        expansion_limit=expansion_limit, ensure_diversity=ensure_diversity, 
        target_col=target_col if ensure_diversity else None):
        
        # get training and testing data for this window
        X_train = df.iloc[train_indices].drop(columns=target_col)
        y_train = df.iloc[train_indices][target_col]
        X_test = df.iloc[test_indices].drop(columns=target_col)
        y_test = df.iloc[test_indices][target_col]

        # train the model
        if isinstance(model, (XGBRegressor, XGBClassifier)):
            model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
        else:
            model.fit(X_train, y_train) # fallback
        
        # check if the model has the predict_proba method (i.e., likely a classifier)
        if hasattr(model, 'predict_proba'):
            # store predicted probabilities of the positive class
            proba = model.predict_proba(X_test)[:, 1]
            model_outputs.extend(list(proba.flatten()))  # ensure it's flattened
        elif hasattr(model, 'predict'):
            # predict for models that support predict (regressors and classifiers without predict_proba)
            predictions = model.predict(X_test)
            model_outputs.extend(list(predictions.flatten()))  # ensure it's flattened
        else:
            raise ValueError("Model does not support required prediction methods.")

        # store true labels for evaluation
        y_true.extend(list(y_test))

    end_time = time.time()
    print(f"Total time taken: {end_time - start_time:.2f} seconds")
    
    if fitted_model:
        return model, model_outputs, y_true
    else:
        return model_outputs, y_true

In [9]:
def train_models_over_grid(df, target_col, initial_train_size, test_size, gap_size,   
                           expansion_limit, model_class, constant_params, explore_params):
    """
    Trains models over a grid of hyperparameters.

    Parameters:
    - df (pd.DataFrame): The dataset to use for training.
    - target_col (str): The name of the target column.
    - initial_train_size (int): Starting size of the training set.
    - test_size (int): Size of the test dataset for each split (LOO cross-validation).
    - gap_size (int): The gap size between training and test datasets. Default is 0.
    - expansion_limit (int): Maximum number of new training observations in expansion.
    - model_class: The class of the model to instantiate.
    - constant_params (dict): Constant parameters for the model.
    - explore_params (dict): Parameters to explore with grid search.

    Returns:
    - dict: A dictionary containing the results for each run.
    """
    import itertools
    import time

    results = {}
    keys, values = zip(*explore_params.items())
    param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    start_time = time.time()

    for i, explore_param in enumerate(param_combinations):
        print('Parameters currently explored:', explore_param)
        
        # instantiate the model with combined parameters
        model = model_class(**constant_params, **explore_param)

        # train over expanding window
        model_outputs, y_true = train_with_expanding_window(
            df=df,
            initial_train_size=initial_train_size,
            test_size=test_size,
            gap_size=gap_size,
            expansion_limit=expansion_limit,
            target_col=target_col,
            model=model
        )
        
        # store outputs and true values in the results dictionary
        results[f"run_{i}"] = {
            "params": {**explore_param},
            "model_outputs": model_outputs,
            "y_true": y_true
        }

    end_time = time.time()
    print(f"Total time taken: {end_time - start_time:.2f} seconds")
    return results

<a name="1.-Target:-Total-Points-(over-/-under)"></a>
# 1. Target: Total Points (over / under)

[Return to top](#Models)

In [10]:
# configuration for expanding window
results = train_models_over_grid(
    model_class=XGBRegressor, # model class
    target_col='TOTAL_PTS', # target column name
    df=pts_sub_fw_features, # data set to use
    initial_train_size=season_22_ngames, # starting size of the training set
    test_size=season_23_ngames,  # leave-one-out (LOO) cross-validation
    gap_size = 0,  # should there be a gap between train and test sets?
    expansion_limit=None, # maximum number of new training observations in expansion
    constant_params={
         'random_state': 599,
         'n_jobs': -1, 
         'objective': 'reg:squarederror',
         'eval_metric': 'rmse',
         'early_stopping_rounds': 20,
         'booster': 'gbtree',
         'n_estimators': 500   # tried: 100, 500, 1000
    },
    explore_params={  
        'learning_rate': [0.01, 0.1, 0.5],      # tried: 0.01, 0.1, 0.5, 1, 2, 3
        'max_depth': [1, 2, 3],            # tried: 1, 2, 3, 4, 5, 6, 8
        'alpha': [5, 7],            # tried: 0.5, 1, 1.5, 2, 3, 4
        'lambda': [10, 12],              # tried: 1, 2, 5, 8, 10, 12
        'gamma': [8, 10]                # tried: 3, 5, 7, 9
    }
)

Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 1, 'alpha': 5, 'lambda': 10, 'gamma': 8}
Total time taken: 1.43 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 1, 'alpha': 5, 'lambda': 10, 'gamma': 10}
Total time taken: 1.37 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 1, 'alpha': 5, 'lambda': 12, 'gamma': 8}
Total time taken: 1.31 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 1, 'alpha': 5, 'lambda': 12, 'gamma': 10}
Total time taken: 1.28 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 1, 'alpha': 7, 'lambda': 10, 'gamma': 8}
Total time taken: 1.28 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 1, 'alpha': 7, 'lambda': 10, 'gamma': 10}
Total time taken: 1.29 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 1, 'alpha': 7, 'lambda': 12, 'gamma': 8}
Total time taken: 1.27 seconds
Parameters curren

In [11]:
# get metrics for each combination of parameter values
results_df = utl.compile_results_to_dataframe(results)

# print best hyperparameter settings
results_df.sort_values(by='average_rmse', ascending=True).head()

,run_id,alpha,average_rmse,gamma,lambda,learning_rate,max_depth,null_rmse
38,run_38,7,19.422,8,12,0.1,2,19.858
39,run_39,7,19.422,10,12,0.1,2,19.858
54,run_54,7,19.425,8,12,0.5,1,19.858
55,run_55,7,19.425,10,12,0.5,1,19.858
31,run_31,7,19.426,10,12,0.1,1,19.858


In [12]:
# get best parameters from validation as dictionary
best_params = utl.get_best_params(results_df, metric='average_rmse')

# save the dictionary to a file
with open('../../hyperparameters/XGB_pts_best_params_selected_fw_features.json', 'w') as json_file:
    json.dump(best_params, json_file, default=utl.handle_non_serializable, indent=4)

<a name="2.-Target:-Difference-in-Points-(plus-/-minus)"></a>
# 2. Target: Difference in Points (plus / minus)

[Return to top](#Models)

In [13]:
# configuration for expanding window
results = train_models_over_grid(
    model_class=XGBRegressor, # model class
    target_col='PLUS_MINUS', # target column name
    df=pm_sub_fw_features, # data set to use
    initial_train_size=season_22_ngames, # starting size of the training set
    test_size=season_23_ngames,  # leave-one-out (LOO) cross-validation
    gap_size = 0,  # should there be a gap between train and test sets?
    expansion_limit=None, # maximum number of new training observations in expansion
    constant_params={
        'random_state': 599,
        'n_jobs': -1,
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'early_stopping_rounds': 20,
        'booster': 'gbtree',
        'n_estimators': 500
    },
    explore_params={
        'learning_rate': [0.1, 0.5, 1],    # tried: 0.1, 0.5, 1, 2
        'max_depth': [1, 2, 3],          # tried: 1, 2, 3, 4
        'alpha': [4, 6],             # tried: 0.1, 0.5, 1, 2, 3, 4
        'lambda': [2, 3],             # tried: 1, 3, 5, 7
        'gamma': [5, 7]               # tried: 5, 7, 9, 11
    }
)

Parameters currently explored: {'learning_rate': 0.1, 'max_depth': 1, 'alpha': 4, 'lambda': 2, 'gamma': 5}
Total time taken: 0.24 seconds
Parameters currently explored: {'learning_rate': 0.1, 'max_depth': 1, 'alpha': 4, 'lambda': 2, 'gamma': 7}
Total time taken: 0.25 seconds
Parameters currently explored: {'learning_rate': 0.1, 'max_depth': 1, 'alpha': 4, 'lambda': 3, 'gamma': 5}
Total time taken: 0.21 seconds
Parameters currently explored: {'learning_rate': 0.1, 'max_depth': 1, 'alpha': 4, 'lambda': 3, 'gamma': 7}
Total time taken: 0.23 seconds
Parameters currently explored: {'learning_rate': 0.1, 'max_depth': 1, 'alpha': 6, 'lambda': 2, 'gamma': 5}
Total time taken: 0.25 seconds
Parameters currently explored: {'learning_rate': 0.1, 'max_depth': 1, 'alpha': 6, 'lambda': 2, 'gamma': 7}
Total time taken: 0.25 seconds
Parameters currently explored: {'learning_rate': 0.1, 'max_depth': 1, 'alpha': 6, 'lambda': 3, 'gamma': 5}
Total time taken: 0.26 seconds
Parameters currently explored: {'l

In [14]:
# get metrics for each combination of parameter values
results_df = utl.compile_results_to_dataframe(results)

# print best hyperparameter settings
results_df.sort_values(by='average_rmse', ascending=True).head()

,run_id,alpha,average_rmse,gamma,lambda,learning_rate,max_depth,null_rmse
38,run_38,6,13.766,5,3,0.5,2,14.254
39,run_39,6,13.766,7,3,0.5,2,14.254
35,run_35,4,13.767,7,3,0.5,2,14.254
34,run_34,4,13.767,5,3,0.5,2,14.254
36,run_36,6,13.789,5,2,0.5,2,14.254


In [15]:
# get best parameters from validation as dictionary
best_params = utl.get_best_params(results_df, metric='average_rmse')

# save the dictionary to a file
with open('../../hyperparameters/XGB_pm_best_params_selected_fw_features.json', 'w') as json_file:
    json.dump(best_params, json_file, default=utl.handle_non_serializable, indent=4)

<a name="3.-Target:-Game-Winner-(moneyline)"></a>
# 3. Target: Game Winner (moneyline)

[Return to top](#Models)

In [16]:
# configuration for expanding window
results = train_models_over_grid(
    model_class=XGBClassifier, # model class
    target_col='GAME_RESULT', # target column name
    df=res_sub_fw_features, # data set to use
    initial_train_size=season_22_ngames, # starting size of the training set
    test_size=season_23_ngames,  # leave-one-out (LOO) cross-validation
    gap_size = 0,  # should there be a gap between train and test sets?
    expansion_limit=None, # maximum number of new training observations in expansion
    constant_params={
        'random_state': 599,
        'n_jobs': -1,
        'objective': 'binary:logistic',
        'eval_metric': 'error',
        'early_stopping_rounds': 20,
        'booster': 'gbtree',
        'n_estimators': 500
    },
    explore_params={
        'learning_rate': [0.01, 0.1, 0.2],    # tried: 0.01, 0.1, 0.5, 1, 2
        'max_depth': [3, 4, 5],             # tried: 1, 2, 3, 4, 5, 6
        'alpha': [4, 5],                    # tried: 1, 2, 3, 4, 5, 6, 7
        'lambda': [10, 12],               # tried: 3, 5, 7, 9, 11, 13, 15, 17, 20, 25
        'gamma': [8, 10]                 # tried: 3, 5, 7, 8, 9, 10, 12, 13, 16
    }
)

Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 3, 'alpha': 4, 'lambda': 10, 'gamma': 8}
Total time taken: 0.16 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 3, 'alpha': 4, 'lambda': 10, 'gamma': 10}
Total time taken: 0.28 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 3, 'alpha': 4, 'lambda': 12, 'gamma': 8}
Total time taken: 0.19 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 3, 'alpha': 4, 'lambda': 12, 'gamma': 10}
Total time taken: 0.26 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 3, 'alpha': 5, 'lambda': 10, 'gamma': 8}
Total time taken: 0.17 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 3, 'alpha': 5, 'lambda': 10, 'gamma': 10}
Total time taken: 0.21 seconds
Parameters currently explored: {'learning_rate': 0.01, 'max_depth': 3, 'alpha': 5, 'lambda': 12, 'gamma': 8}
Total time taken: 0.15 seconds
Parameters curren

In [17]:
# get metrics for each combination of parameter values
results_df = utl.compile_results_to_dataframe(results)

# print best hyperparameter settings
results_df.sort_values(by='average_accuracy', ascending=False).head()

,run_id,alpha,average_accuracy,average_f1_score,baseline_accuracy,gamma,lambda,learning_rate,max_depth,overall_auc,pred_labels
39,run_39,5,0.624,0.714,0.573,10,12,0.1,4,0.616,"[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,..."
29,run_29,5,0.624,0.714,0.573,10,10,0.1,3,0.616,"[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,..."
31,run_31,5,0.624,0.714,0.573,10,12,0.1,3,0.616,"[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,..."
47,run_47,5,0.624,0.714,0.573,10,12,0.1,5,0.616,"[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,..."
45,run_45,5,0.624,0.714,0.573,10,10,0.1,5,0.616,"[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,..."


In [18]:
# get best parameters from validation as dictionary
best_params = utl.get_best_params(results_df, metric='average_accuracy')

# save the dictionary to a file
with open('../../hyperparameters/XGB_res_best_params_selected_fw_features.json', 'w') as json_file:
    json.dump(best_params, json_file, default=utl.handle_non_serializable, indent=4)